In [2]:
pip install kagglehub


   ---------------------------------------- 0/4 [tqdm]
   ---------------------------------------- 0/4 [tqdm]
   ---------- ----------------------------- 1/4 [charset_normalizer]
   ---------- ----------------------------- 1/4 [charset_normalizer]
   -------------------- ------------------- 2/4 [requests]
   ------------------------------ --------- 3/4 [kagglehub]
   ------------------------------ --------- 3/4 [kagglehub]
   ---------------------------------------- 4/4 [kagglehub]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import shutil
import kagglehub

# Download latest version
path = kagglehub.dataset_download("alejandroczernikier/properati-argentina-dataset")

print("Path to dataset files:", path)

dest = "../data/"
os.makedirs(dest, exist_ok=True)
for file in os.listdir(path):
    shutil.move(os.path.join(path, file), os.path.join(dest, file))

Path to dataset files: C:\Users\juanf\.cache\kagglehub\datasets\alejandroczernikier\properati-argentina-dataset\versions\1
